In [1]:
import os
import subprocess

def process_videos_in_folder(folder_path):
    for file_name in os.listdir(folder_path):
        if file_name.startswith("filtered_"):
            print(f"El video {file_name} ya fue procesado")
            continue
        if file_name.endswith(".mp4"):
            input_path = os.path.join(folder_path, file_name)
            output_path = os.path.join(folder_path, f"filtered_{file_name}")
            
            # Comando ffmpeg
            command = [
                "ffmpeg", 
                "-i", input_path, 
                "-vf", r"select=not(mod(n\,3))", 
                "-vsync", "vfr", 
                "-c:v", "libx264", 
                output_path
            ]
            
            try:
                subprocess.run(command, check=True)
                print(f"Video procesado: {file_name}")
                os.remove(input_path)
            except subprocess.CalledProcessError as e:
                print(f"Error procesando el video {file_name}: {e}")

In [2]:
import sys

def create_base_path(inContainer: bool):
    if inContainer:
        base_path = "/" + os.getcwd().split("/")
    else:
        base_path = os.getcwd().split("\\")
    return base_path
base_path = create_base_path(inContainer=False)
project_directory = r""
for part in base_path:
    if part != "Modules":
        project_directory += part
        project_directory += r"/"
modules_directory = r"Modules"

modules_path = project_directory + modules_directory

sys.path.append(modules_path)
print(project_directory)
print(os.listdir(project_directory))

c:/Users/matia/OneDrive/Escritorio/SignAI-ML/AI-Module/
['BERT', 'CNN-LSTM', 'Extra', 'Modules', 'Resources', 'Tests', '__init__.py']


In [10]:
#root_vids = project_directory + "Resources/Videos/raw_videos"
root_vids = r"E:\Sprinfil\how2sign\raw_videos\usedvideos"
root_translation_csv = project_directory + "Resources/Translations"

In [4]:
def obtain_paths(root, extension):
    paths = []

    for filename in os.listdir(root):
        if filename.endswith(extension):
            paths.append(os.path.join(root, filename))
    return paths

In [5]:
import KeyFrameExtractorClass
import Points2VecClass
import cv2
import VideoFormaterClass
import pandas as pd

In [11]:
vids = obtain_paths(root_vids, '.mp4')[:500]
translation_csv = obtain_paths(root_translation_csv, '.csv')[1]
print("vids:" + str(len(vids)))
print("translation_csv: " + str(translation_csv))

vids:500
translation_csv: c:/Users/matia/OneDrive/Escritorio/SignAI-ML/AI-Module/Resources/Translations\how2sign_train.csv


In [7]:
def make_df_video(video_paths, csv_path, type):
    kfe = KeyFrameExtractorClass.KeyFrameExtractor()
    videoFormater = VideoFormaterClass.VideoFormater()
    normalizer = Points2VecClass.Point2Vec(4)
    translationDf = videoFormater.csvToTranslationDf(csv_path)
    if not os.path.exists(project_directory + f"Resources/Datasets/{type}"):
        os.mkdir(project_directory + f"Resources/Datasets/{type}")
    for index, path in enumerate(video_paths):
        print("Processing video: ", index)
        video = cv2.VideoCapture(path)
        file_name = path.split(os.path.sep)[-1].split(".mp4")[0].split("filtered_")[1]
        if (file_name + ".csv") in os.listdir(project_directory + f"Resources/Datasets/{type}"):
            print(f"El video {index} ya fue procesado")
            continue
        translation_filter = translationDf["VIDEO_NAME"] == file_name
        translation = translationDf[translation_filter]["SENTENCE"].iloc[0]
        video_points = kfe.extractKeyFrames(return_frame=False, draw=False, video=video)
        landmarks = normalizer.land2vec(video_points)
        cant_keyframes = len(video_points)
        dict = {"points": landmarks, "translation": translation, "id": index, "len_keyframes": cant_keyframes}
        dataFrame = videoFormater.formatVideo(dict)
        videoFormater.concatAndExportVideos(dataFrame, project_directory + f"Resources/Datasets/{type}/{file_name}.csv")
    return project_directory + f"Resources/Datasets/{type}"

In [8]:
def processVideos(paths, csv_path, folder_name, root_vids):
    print(paths, csv_path, folder_name)
    process_videos_in_folder(root_vids)
    csvs_path = make_df_video(paths, csv_path, folder_name)
    return csvs_path

In [12]:
csvs_path = processVideos(vids, translation_csv, "videos", root_vids)

['E:\\Sprinfil\\how2sign\\raw_videos\\usedvideos\\filtered_1TlNisOTK9Q-5-rgb_front.mp4', 'E:\\Sprinfil\\how2sign\\raw_videos\\usedvideos\\filtered_1_N0Aq04tRA-8-rgb_front.mp4', 'E:\\Sprinfil\\how2sign\\raw_videos\\usedvideos\\filtered_bpyAuV3jNIc-8-rgb_front.mp4', 'E:\\Sprinfil\\how2sign\\raw_videos\\usedvideos\\filtered_FWxpecaJ2mA-5-rgb_front.mp4', 'E:\\Sprinfil\\how2sign\\raw_videos\\usedvideos\\filtered_0IsoobG67PY-5-rgb_front.mp4', 'E:\\Sprinfil\\how2sign\\raw_videos\\usedvideos\\filtered_CsdpA_b5P8Q-5-rgb_front.mp4', 'E:\\Sprinfil\\how2sign\\raw_videos\\usedvideos\\filtered_aSjHu-I75bE-5-rgb_front.mp4', 'E:\\Sprinfil\\how2sign\\raw_videos\\usedvideos\\filtered_1Qz0SIXS3no-5-rgb_front.mp4', 'E:\\Sprinfil\\how2sign\\raw_videos\\usedvideos\\filtered_a6xKFGeHX4A-5-rgb_front.mp4', 'E:\\Sprinfil\\how2sign\\raw_videos\\usedvideos\\filtered_0pryZawZCJI-5-rgb_front.mp4', 'E:\\Sprinfil\\how2sign\\raw_videos\\usedvideos\\filtered_EUX-PNdmh-g-5-rgb_front.mp4', 'E:\\Sprinfil\\how2sign\\raw_vi

c:\Users\matia\OneDrive\Escritorio\SignAI-ML\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Threshold: $0.05518374001065088 
 Hands size: $0.00847977710331449
Threshold: $0.03063803035936638 
 Hands size: $0.00021446422390563363
Threshold: $0.030326467599357504 
 Hands size: $0.00010973704001297111
Threshold: $0.03027117646316943 
 Hands size: $9.115177056531462e-05
Threshold: $0.03016511088270138 
 Hands size: $5.549946048721832e-05
Threshold: $0.03015204092555937 
 Hands size: $5.110619668002032e-05
Threshold: $0.030201646861386183 
 Hands size: $6.778046510825889e-05
Threshold: $0.030150424642602175 
 Hands size: $5.05629081896079e-05
Threshold: $0.03009285401752812 
 Hands size: $3.1211435195484714e-05
Threshold: $0.03012188884166908 
 Hands size: $4.0971040872828665e-05
Threshold: $0.030133649991430006 
 Hands size: $4.492436905078197e-05
Threshold: $0.030083000524388753 
 Hands size: $2.789933645199211e-05
Threshold: $0.030107704043354916 
 Hands size: $3.620304092591089e-05
Threshold: $0.030065437424329905 
 Hands size: $2.199577314028775e-05
Processing video:  288
Thr

In [19]:
def joincsvs(path):
    csvs = obtain_paths(path, '.csv')
    final_df = pd.DataFrame()
    for index, csv in enumerate(csvs):
        print("Processing csv: ", index)
        for chunk in pd.read_csv(csv, chunksize=10000):
            final_df = pd.concat([final_df, chunk], ignore_index=True)
    videoFormater = VideoFormaterClass.VideoFormater()
    videoFormater.concatAndExportVideos(final_df, project_directory + "Resources/Datasets/how2sign.csv")

In [20]:
csvs_path = project_directory + f"Resources/Datasets/videos"
joincsvs(csvs_path)

Processing csv:  0
Processing csv:  1
Processing csv:  2
Processing csv:  3
Processing csv:  4
Processing csv:  5
Processing csv:  6
Processing csv:  7
Processing csv:  8
Processing csv:  9
Processing csv:  10
Processing csv:  11
Processing csv:  12
Processing csv:  13
Processing csv:  14
Processing csv:  15
Processing csv:  16
Processing csv:  17
Processing csv:  18
Processing csv:  19
Processing csv:  20
Processing csv:  21
Processing csv:  22
Processing csv:  23
Processing csv:  24
Processing csv:  25
Processing csv:  26
Processing csv:  27
Processing csv:  28
Processing csv:  29
Processing csv:  30
Processing csv:  31
Processing csv:  32
Processing csv:  33
Processing csv:  34
Processing csv:  35
Processing csv:  36
Processing csv:  37
Processing csv:  38
Processing csv:  39
Processing csv:  40
Processing csv:  41
Processing csv:  42
Processing csv:  43
Processing csv:  44
Processing csv:  45
Processing csv:  46
Processing csv:  47
Processing csv:  48
Processing csv:  49
Processing